In [2]:
import pandas as pd


In [12]:
df_cz = pd.read_excel("data/cz-population.xlsx", header=3)


In [20]:
df_cz = df_cz.dropna()

df_cz = df_cz[pd.to_numeric(df_cz['celkem'], errors='coerce').notnull()]

# Step 4: Convert 'celkem' to numeric for summation
df_cz['celkem'] = pd.to_numeric(df_cz['celkem'])
df_cz.rename(columns={'Unnamed: 2': 'name_obec'}, inplace=True)
# Step 5: Calculate and print the sum of 'celkem'
total_population = df_cz['celkem'].sum()
print("Sum of population in 'celkem' column:", total_population)

Sum of population in 'celkem' column: 10900555


In [34]:
def get_city(station_name, cities):
    possible_cities = [c for c in cities if station_name.startswith(c)]
    if possible_cities:
        return max(possible_cities, key=len)
    return None

In [21]:
city_population = dict(zip(df_cz['name_obec'], df_cz['celkem']))
cities = list(city_population.keys())

In [22]:
import json
from shapely import to_geojson
from shapely.geometry import shape
from networkx.readwrite import json_graph
def graph_from_geojson(path, directed=False, multigraph=True):
    """
    Read a JSON file at 'path' produced by graph_to_geojson and
    reconstruct the original NetworkX graph with Shapely geometries.
    """
    # 1. Load the raw node-link dict
    with open(path) as f:
        data = json.load(f)

    # 2. Convert each node's GeoJSON dict back into a Shapely geometry
    for node_obj in data.get("nodes", []):
        geom_dict = node_obj.get("geometry")
        if isinstance(geom_dict, dict) and "type" in geom_dict:
            node_obj["geometry"] = shape(geom_dict)  # inverse of to_geojson :contentReference[oaicite:1]{index=1}

    # 3. (Optional) If you stored edge geometries similarly, undo those too
    for edge_obj in data.get("links", data.get("edges", [])):
        geom_dict = edge_obj.get("geometry")
        if isinstance(geom_dict, dict) and "type" in geom_dict:
            edge_obj["geometry"] = shape(geom_dict)

    # 4. Rebuild the NetworkX graph (with all attrs, including restored geometries)
    G = json_graph.node_link_graph(
        data,
        directed=directed,
        multigraph=multigraph
    )  # rebuilds Graph from node-link format :contentReference[oaicite:2]{index=2}

    return G


In [30]:
print(cities)

['Praha', 'Benešov', 'Bernartice', 'Bílkovice', 'Blažejovice', 'Borovnice', 'Bukovany', 'Bystřice', 'Ctiboř', 'Čakov', 'Čechtice', 'Čerčany', 'Červený Újezd', 'Český Šternberk', 'Čtyřkoly', 'Děkanovice', 'Divišov', 'Dolní Kralovice', 'Drahňovice', 'Dunice', 'Heřmaničky', 'Hradiště', 'Hulice', 'Hvězdonice', 'Chářovice', 'Chleby', 'Chlístov', 'Chlum', 'Chmelná', 'Chocerady', 'Choratice', 'Chotýšany', 'Chrášťany', 'Jankov', 'Javorník', 'Ješetice', 'Kamberk', 'Keblov', 'Kladruby', 'Kondrac', 'Kozmice', 'Krhanice', 'Krňany', 'Křečovice', 'Křivsoudov', 'Kuňovice', 'Lešany', 'Libež', 'Litichovice', 'Loket', 'Louňovice pod Blaníkem', 'Lštění', 'Maršovice', 'Mezno', 'Miličín', 'Miřetice', 'Mnichovice', 'Mrač', 'Načeradec', 'Nespeky', 'Netvořice', 'Neustupov', 'Neveklov', 'Olbramovice', 'Ostrov', 'Ostředek', 'Pavlovice', 'Petroupim', 'Popovice', 'Poříčí nad Sázavou', 'Postupice', 'Pravonín', 'Přestavlky u Čerčan', 'Psáře', 'Pyšely', 'Rabyně', 'Radošovice', 'Rataje', 'Ratměřice', 'Řehenice', 'Řim

In [ ]:
G = graph_from_geojson("selected_fixed_graphs\cze-railroad-network.json")
node_to_city = {}
for node in G.nodes:
    city = get_city(node, cities)
    if city:
        node_to_city[node] = city
    else:
        print(f"Warning: No city match for {node}")
        # Option: Assign a default city or small weight later


In [46]:
from collections import Counter
# Count stations per city
city_station_count = Counter(node_to_city.values())
# Assign weights to nodes
node_weights = {}
total_population_used = 0
for node, city in node_to_city.items():
    P_C = city_population[city]  # City population
    N_C = city_station_count[city]  # Number of stations in city
    w_node = P_C / N_C  # Weight for this station
    total_population_used += w_node
    node_weights[node] = w_node
    
print(f"Total population used: {total_population_used}")

# Optional: Add weights to graph as node attributes
for node in G.nodes:
    if node in node_weights:
        G.nodes[node]['pop'] = node_weights[node]
    else:
        total_population_used += 1000
        G.nodes[node]['pop'] = 1000 # For unmatched stations

for node in G.nodes:
    G.nodes[node]['prob'] = G.nodes[node]['pop']/total_population_used
    

Total population used: 7179048.000000003


# Run for Czechia

In [ ]:
import numpy as np
from geopy.distance import geodesic
import networkx as nx
import pandas as pd
from tqdm import tqdm
import time

# Load the graph (adjust path as needed)

# Step 1: Precompute nodes and probabilities
nodes = list(G.nodes)
probabilities = [G.nodes[node]['prob'] for node in nodes]
num_iterations = 1_000_000

ratios = []
valid_pairs = 0  

start_time = time.time()
for _ in tqdm(range(num_iterations), desc="Processing iterations"):
    # Select two distinct nodes
    u, v = np.random.choice(nodes, size=2, replace=False, p=probabilities)
    
    # Calculate straight-line distance
    point_u = G.nodes[u]['geometry']
    point_v = G.nodes[v]['geometry']
    coords_u = point_u.coords[0]  # (longitude, latitude)
    coords_v = point_v.coords[0]
    latlon_u = (coords_u[1], coords_u[0])  # (latitude, longitude)
    latlon_v = (coords_v[1], coords_v[0])
    straight_line_distance = geodesic(latlon_u, latlon_v).meters
    
    # Calculate shortest path distance
    try:
        shortest_path_distance = nx.shortest_path_length(G, source=u, target=v, weight='distance')
        # Compute ratio (shortest path distance / straight-line distance)
        if straight_line_distance > 0:  # Avoid division by zero
            ratio = shortest_path_distance / straight_line_distance
            ratios.append(ratio)
            valid_pairs += 1
    except nx.NetworkXNoPath:
        # Skip pairs with no path (or assign ratio = inf if desired)
        continue

results_df = pd.DataFrame({'ratio': ratios})
output_path = 'czech_ratios.csv'
results_df.to_csv(output_path, index=False)

print(f"Total iterations: {num_iterations}")
print(f"Valid pairs (with path): {valid_pairs}")
print(f"Invalid pairs (no path): {num_iterations - valid_pairs}")
if ratios:
    print(f"Mean ratio: {np.mean(ratios):.4f}")
    print(f"Median ratio: {np.median(ratios):.4f}")
    print(f"Min ratio: {np.min(ratios):.4f}")
    print(f"Max ratio: {np.max(ratios):.4f}")
else:
    print("No valid paths found.")
print(f"Results saved to: {output_path}")
print(f"Time taken: {time.time() - start_time:.2f} seconds")

Processing iterations: 100%|██████████| 1000000/1000000 [23:05<00:00, 721.73it/s] 


Total iterations: 1000000
Valid pairs (with path): 967501
Invalid pairs (no path): 32499
Mean ratio: 1.3393
Median ratio: 1.2708
Min ratio: 1.0000
Max ratio: 38.3310
Results saved to: czech_ratios.csv
Time taken: 1386.68 seconds


In [53]:
df_sk = pd.read_csv("data\sk_population.csv")
display(df_sk)
sum(df_sk["population"])

,name,link,region,district,region_historical,elevation,coordinate_x,coordinate_y,area,population,first_mentioned
0,Ábelová,/wiki/%C3%81belov%C3%A1,Banskobystrický kraj,Lučenec,Novohrad,445.0,48.410833,19.432222,52.17,196,1275
1,Abovce,/wiki/Abovce,Banskobystrický kraj,Rimavská Sobota,Gemer,159.0,48.318889,20.336944,8.20,626,1339
2,Abrahám,/wiki/Abrah%C3%A1m_(okres_Galanta),Trnavský kraj,Galanta,undefined,125.0,48.250000,17.616667,15.78,1055,1231
3,Abrahámovce,/wiki/Abrah%C3%A1movce_(okres_Bardejov),Prešovský kraj,Bardejov,Šariš,267.0,49.161667,21.342222,5.90,341,1427
4,Abrahámovce,/wiki/Abrah%C3%A1movce_(okres_Ke%C5%BEmarok),Prešovský kraj,Kežmarok,undefined,739.0,49.044669,20.437262,6.65,269,1286
...,...,...,...,...,...,...,...,...,...,...,...
2885,Žitavce,/wiki/%C5%BDitavce,Nitriansky kraj,Nitra,undefined,141.0,48.201690,18.299638,8.29,396,1232
2886,Žitná-Radiša,/wiki/%C5%BDitn%C3%A1-Radi%C5%A1a,Trenčiansky kraj,Bánovce nad Bebravou,Horná Nitra,275.0,48.766500,18.344400,17.77,443,1295
2887,Žlkovce,/wiki/%C5%BDlkovce,Trnavský kraj,Hlohovec,Dolné Považie,145.0,48.466667,17.716667,7.94,660,1229
2888,Župčany,/wiki/%C5%BDup%C4%8Dany,Prešovský kraj,Prešov,Šariš,315.0,49.016667,21.166667,8.49,1690,1248


5457936

In [54]:
city_population = dict(zip(df_sk['name'], df_sk['population']))
cities = list(city_population.keys())


In [56]:
G = graph_from_geojson("selected_fixed_graphs\svk-railroad-network.json")
node_to_city = {}
for node in G.nodes:
    city = get_city(node, cities)
    if city:
        node_to_city[node] = city
    else:
        print(f"Warning: No city match for {node}")
        # Option: Assign a default city or small weight later


c:\Users\pazera\anaconda3\envs\starter\Lib\site-packages\networkx\readwrite\json_graph\node_link.py:287: FutureWarning: 
The default value will be changed to `edges="edges" in NetworkX 3.6.

To make this warning go away, explicitly set the edges kwarg, e.g.:

  nx.node_link_graph(data, edges="links") to preserve current behavior, or
  nx.node_link_graph(data, edges="edges") for forward compatibility.
  warnings.warn(


In [57]:
from collections import Counter
# Count stations per city
city_station_count = Counter(node_to_city.values())
# Assign weights to nodes
node_weights = {}
total_population_used = 0
for node, city in node_to_city.items():
    P_C = city_population[city]  # City population
    N_C = city_station_count[city]  # Number of stations in city
    w_node = P_C / N_C  # Weight for this station
    total_population_used += w_node
    node_weights[node] = w_node
    
print(f"Total population used: {total_population_used}")

# Optional: Add weights to graph as node attributes
for node in G.nodes:
    if node in node_weights:
        G.nodes[node]['pop'] = node_weights[node]
    else:
        total_population_used += 1000
        G.nodes[node]['pop'] = 1000 # For unmatched stations

for node in G.nodes:
    G.nodes[node]['prob'] = G.nodes[node]['pop']/total_population_used
    

Total population used: 3096147.9999999995


# Run for Slovakia

In [ ]:
import numpy as np
from geopy.distance import geodesic
import networkx as nx
import pandas as pd
from tqdm import tqdm
import time

# Load the graph (adjust path as needed)

# Step 1: Precompute nodes and probabilities
nodes = list(G.nodes)
probabilities = [G.nodes[node]['prob'] for node in nodes]
num_iterations = 1_000_000

ratios = []
valid_pairs = 0  

start_time = time.time()
for _ in tqdm(range(num_iterations), desc="Processing iterations"):
    # Select two distinct nodes
    u, v = np.random.choice(nodes, size=2, replace=False, p=probabilities)
    
    # Calculate straight-line distance
    point_u = G.nodes[u]['geometry']
    point_v = G.nodes[v]['geometry']
    coords_u = point_u.coords[0]  # (longitude, latitude)
    coords_v = point_v.coords[0]
    latlon_u = (coords_u[1], coords_u[0])  # (latitude, longitude)
    latlon_v = (coords_v[1], coords_v[0])
    straight_line_distance = geodesic(latlon_u, latlon_v).meters
    
    # Calculate shortest path distance
    try:
        shortest_path_distance = nx.shortest_path_length(G, source=u, target=v, weight='distance')
        # Compute ratio (shortest path distance / straight-line distance)
        if straight_line_distance > 0:  # Avoid division by zero
            ratio = shortest_path_distance / straight_line_distance
            ratios.append(ratio)
            valid_pairs += 1
    except nx.NetworkXNoPath:
        # Skip pairs with no path (or assign ratio = inf if desired)
        continue

results_df = pd.DataFrame({'ratio': ratios})
output_path = 'Slovak_ratios.csv'
results_df.to_csv(output_path, index=False)

print(f"Total iterations: {num_iterations}")
print(f"Valid pairs (with path): {valid_pairs}")
print(f"Invalid pairs (no path): {num_iterations - valid_pairs}")
if ratios:
    print(f"Mean ratio: {np.mean(ratios):.4f}")
    print(f"Median ratio: {np.median(ratios):.4f}")
    print(f"Min ratio: {np.min(ratios):.4f}")
    print(f"Max ratio: {np.max(ratios):.4f}")
else:
    print("No valid paths found.")
print(f"Results saved to: {output_path}")
print(f"Time taken: {time.time() - start_time:.2f} seconds")

Processing iterations: 100%|██████████| 1000000/1000000 [20:59<00:00, 793.83it/s]


Total iterations: 1000000
Valid pairs (with path): 971319
Invalid pairs (no path): 28681
Mean ratio: 1.7776
Median ratio: 1.3172
Min ratio: 1.0000
Max ratio: 375.0650
Results saved to: Slovak_ratios.csv
Time taken: 1262.69 seconds


In [ ]:
# https://github.com/adammertel/municipalities-slovakia/blob/master/out/municipalities-slovakia.csv
# slovakia population data source 
# https://csu.gov.cz/produkty/pocet-obyvatel-v-obcich-9vln2prayv
# czechia population data source 